# Problem Statement 

The problem statement is to detect intrusions from a tcp dump.The data is provided in csv files so no feature extraction is involved 

In [28]:
from data import fetch_data 
from data import feature_engineering
from models.model import Model
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import os
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from Para_learn import paralearn
import pickle
from flask import request
from cross_val import k_fold_crossval
%matplotlib inline 
root=os.getcwd()

In [29]:
x_train,y_train,x_test,y_test = fetch_data(root,remove_duplicates=True,binary=True)

Datasets loaded :)
The dimensions of the training dataset is (145583, 41)
The dimensions of test dataset is (311028, 41)
The taken to load data is 4.65 secs


In [30]:
x_train = feature_engineering(x_train,do_normalization=False)
x_test = feature_engineering(x_test,do_normalization=False)
print('Shape of training data after feature engineering is {}'.format(x_train.shape))
print ('Shape of test data after feature engineering is {}'.format(x_test.shape))

Shape of training data after feature engineering is (145583, 45)
Shape of test data after feature engineering is (311028, 45)


In [ ]:
y_train.value_counts()

# Sampling 

# Modelling in progress

In [4]:

clf=Model(x_train,y_train)


In [5]:
#NAIVE BAYES
nb=clf.mnbayes(x_train,y_train)
predict_train_nb=nb.predict(x_train)
conf_matrix_nb=confusion_matrix(list(y_train),(predict_train_nb).round())
f1_score_train_nb=f1_score(list(y_train),(predict_train_nb).round())
print (conf_matrix_nb)
print('The f1 score for NAIVE BAYES model on training data is {}'.format(f1_score_train_nb))


[[77644 10170]
 [18535 39234]]
The f1 score for NAIVE BAYES model on training data is 0.7321620184188181


In [6]:
cross_val_score_nb=k_fold_crossval(nb,x_train,y_train,6)

f1 score for 1 iteration................. 0.6481
f1 score for 2 iteration................. 0.6467
f1 score for 3 iteration................. 0.6550
f1 score for 4 iteration................. 0.6318
f1 score for 5 iteration................. 0.8785
f1 score for 6 iteration................. 0.6561


In [ ]:
#SVM 
svm=clf.svm(x_train,y_train)
predict_train_svm=svm.predict(x_train)
conf_matrix_svm=confusion_matrix(list(y_train),(predict_train_svm).round())
f1_score_train_svm=f1_score(list(y_train),(predict_train_svm).round())
print (conf_matrix_svm)
print('The f1 score for SUPPORT VECTOR MACHINES model on training data is {}'.format(f1_score_train_svm))


In [ ]:
cross_val_score_svm=k_fold_crossval(svm,x_train,y_train,6)

In [ ]:
#XGBOOST 
xgb,x_dtrain,x_dtest=clf.xgboost(x_train,y_train,x_test)
predict_train_xgb=xgb.predict(x_dtrain)
conf_matrix_xgb=confusion_matrix(list(y_train),(predict_train_xgb).round())
f1_score_train_xgb=f1_score(list(y_train),(predict_train_xgb).round())
print (conf_matrix_xgb)
print('The f1 score for XGBOOST model on training data is {}'.format(f1_score_train_xgb))


In [ ]:
#Checking whether this is a overfit on the training data 


In [8]:
#RANDOMFOREST 
rf=clf.randforest(x_train,y_train)
predict_train_rf=rf.predict(x_train)
conf_matrix_rf=confusion_matrix(list(y_train),(predict_train_rf).round())
f1_score_train_rf=f1_score(list(y_train),(predict_train_rf).round())
print (conf_matrix_rf)
print('The f1 score for RANDOM FOREST model on training data is {:.4f}'.format(f1_score_train_rf))


[[87814     0]
 [   17 57752]]
The f1 score for RANDOM FOREST model on training data is 0.9999


In [9]:
# Cross validation to check whether the model is overfitting or not 
cross_val_score_rf=k_fold_crossval(rf,x_train,y_train,6)


f1 score for 1 iteration................. 0.9993
f1 score for 2 iteration................. 0.9992
f1 score for 3 iteration................. 0.9990
f1 score for 4 iteration................. 0.9988
f1 score for 5 iteration................. 0.9993
f1 score for 6 iteration................. 0.9989


# Model Selection 


After doing hyperparameter optimization for different models on the training data and computing a metric evaluating the generalization of a particular model,we can select which model will be our best bet for generalizing over to the unseen data 

# Hitting the API 

In [10]:
#Random forest is the one. Lets save this model somewhere and hit the api to see what happens 
joblib.dump(rf,'random_forest.pkl')

['random_forest.pkl']

In [11]:
# Hitting the api 
root

'/home/anurag/Desktop/DS_Training'

In [20]:
import pickle

In [27]:
pickle.load(open('rf_test.pkl','rb'))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=2, oob_score=False,
            random_state=12345, verbose=0, warm_start=False)

In [26]:
pickle.dump(rf,open('rf_test.pkl','wb'),protocol=2)